# TP 3 de Statistique descriptive avec SAS

Objectifs de la séance : 
* blabbla
* blablabla

**Question 1** : Créez une bibliothèque *source* qui contiendra les données en lecture seule. Créez également une bibliothèque *resultat* qui contiendra les résultats de vos exercices

In [ ]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP3/source" ACCESS = READONLY;
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP3/resultat";

**Question 2** : Affichez les dix premières lignes de la table *equipement* et affichez des informations générales dessus. 

In [ ]:
PROC PRINT DATA = source.equipement(obs = 10);
RUN;

In [ ]:
PROC CONTENTS DATA = source.equipement;
RUN;

**Question 3** : La variable COMINSEE correspond au code géographique de la commune (il est différent du code postal). Néanmoins, ce code est censé être sur cinq positions (i.e de la forme xxxxx). Est-ce le cas pour toutes les communes ? Utilisez un format prédéfini adapté pour afficher correctement les codes géographiques. 

La table après application du format se nommera *equipement* et sera stockée dans la bibliothèque *resultat*.

**Réponse** : L'idée est de faire utiliser le format *Zn.*

In [ ]:
DATA resultat.equipement;
SET source.equipement;
FORMAT COMINSEE Z5.;
RUN;

**Question** : La table *libelle* contient deux variables permettant de faire le lien entre le numéro d'une activité (*Numero*) et le nom de l'activité (*Libelle*). À partir de cette table, créez le format *nomactivite* qui à un numéro d'activité associera le nom de l'activité.

In [ ]:
DATA format_lien;
SET source.libelle;
fmtname = "nomactivite";
RENAME numero = start
libelle = label;
RUN;

PROC FORMAT CNTLIN = format_lien;
RUN;

**Question** 